<a href="https://colab.research.google.com/github/paytons3/Fung/blob/main/MEng_Section_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MEng Section Matching Project
By John Jun and Payton Steiner

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

## Data Preparation

In [ ]:
df = pd.read_csv('/content/MEng input data.csv')
df = df.iloc[0:115]

In [ ]:
len(df)

115

Reset index to project id

In [ ]:
df_copy = df.copy()
df_copy.set_index('Please indicate your Project ID#. ', inplace=True)

Get df with only 295 section prefs

In [ ]:
#295 df
df_295 = df_copy.copy()
df_295 = pd.concat([df_copy.iloc[:, 0:7], df_295.iloc[0:, 7:21]], axis = 1)

#295 sections only
df_295_sections = df_295.iloc[:, 7:]
df_295_sections.head()

,"For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 3: WED 10:00 - 10:59am; Instructor: M. Bauer]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 4: WED 11:00 - 11:59am; Instructor: M. Bauer]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 5: WED 1:00 - 1:59pm; Instructor: M. Bauer]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 6: THURS 10:00 - 10:59am; Instructor: M. Bauer]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 7: THURS 11:00 - 11:59am; Instructor: M. Bauer]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 8: WED 11:00 - 11:59am; Instructor: J. Caleshu]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 9: WED 1:00 - 1:59pm; Instructor: J. Caleshu]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 10: MON 10:00 - 10:59am; Instructor: T. Fitzpatrick]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 11: MON 12:00 - 12:59pm; Instructor: T. Fitzpatrick]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 12: MON 1:00 - 1:59pm; Instructor: T. Fitzpatrick]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 13: TUES 9:00 - 9:59am; Instructor: S. Houlihan]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 14: TUES 10:00 - 10:59am; Instructor: S. Houlihan]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 15: TUES 11:00 - 11:59am; Instructor: S. Houlihan]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 16: MON 11:00 - 11:59am; Instructor: M. Kolb]"
Please indicate your Project ID#.,,,,,,,,,,,,,,
164,5th choice,4th choice,2nd choice,NaN,NaN,3rd choice,1st choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,NaN,1st choice,NaN,3rd choice,4th choice,2nd choice,NaN,NaN,NaN,NaN,NaN,NaN,5th choice,NaN
130,NaN,1st choice,NaN,3rd choice,NaN,4th choice,NaN,NaN,NaN,NaN,NaN,2nd choice,NaN,NaN
094,NaN,NaN,NaN,1st choice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2nd choice,NaN,NaN
088,3rd choice,NaN,NaN,2nd choice,NaN,NaN,NaN,NaN,5th choice,4th choice,NaN,1st choice,NaN,NaN


Get df with only 270k section prefs

In [ ]:
#270k df
df_270k = df_copy.copy()
df_270k = pd.concat([df_copy.iloc[:, 0:7], df_270k.iloc[0:, 21:]], axis = 1)

#270k sections only
df_270k_sections = df_270k.iloc[:, 7:]
df_270k_sections.head()

,"For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101A: TUES 10:00 - 10:30am; Instructor: A. Balaraman]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101B: TUES 10:30 - 11:00am; Instructor: A. Balaraman]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101C: TUES 11:30am - 12:00pm; Instructor: A. Balaraman]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101D: TUES 12:00 - 12:30pm; Instructor: A. Balaraman]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 102A: FRI 12:00 - 12:30pm; Instructor: P. Deb]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 102B: FRI 12:30 - 1:00pm; Instructor: P. Deb]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 102C: FRI 1:00 - 1:30pm; Instructor: P. Deb]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 102D: FRI 1:30 - 2:00pm; Instructor: P. Deb]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 103A: TUES 2:30 - 3:00pm; Instructor: L. Edwards]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 103B: TUES 3:00 - 3:30pm; Instructor: L. Edwards]",...,"For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 112C: TUES 12:30 - 1:00pm; Instructor: H. Weller]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 112D: TUES 1:15 - 1:45pm; Instructor: H. Weller]","For ENGIN295 - Communications for Engineering Leaders, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [N/A (choose for choices not possible)]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [N/A (choose for choices not possible)]",Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85
Please indicate your Project ID#.,,,,,,,,,,,,,,,,,,,,,
164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5th choice,5th choice,NaN,NaN,NaN,NaN,NaN,NaN
094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creating Classes

In [ ]:
#Creating class for each discussion sections
class Section:

  def __init__(self, sectionname, capacity, currTime):
    self.name = sectionname
    self.capacity = capacity
    self.numStudents = 0
    self.teams = [] # could be a different datastructure
    self.worstPref = None # gets assigned when first team is assigned and then updated if a new team has a worse pref
    self.worstPrefTeam = None
    self.currTime = currTime

  def worstPrefCheck(self, team):
    """
    If worstPref hasn't been assigned yet, assign this team as the team with the worst
    preference for the section. If it has been assigned but this team has a worse
    preference than the current worst, reassign the worst preference team to this one.
    Else, do nothing.
    """
    if self.worstPref == None:
      self.assignWorstPref(team.currPref, team)
    elif team.currPref >= self.worstPref:
      self.assignWorstPref(team.currPref, team)

  def assignWorstPref(self, pref, team):
    """
    Assign pref as this section's worst preference number
    and team as this section's team with worst preference.
    Worst refers to the preference number being the largest.
    """
    self.worstPref = pref
    self.worstPrefTeam = team

  def seekOtherWorst(self, sizeRequirement):
    """
    Find a team that has the same or worse preference than the current team
    with a team size bigger than the current team. If found, make that
    team the new worst team and preference for this section.
    """
    current_worst = self.worstPrefTeam
    for team in self.teams:
      if team.size > sizeRequirement and team.currPref >= self.worstPref:
        self.assignWorstPref(team.currPref, team)
    if self.worstPrefTeam != current_worst:
      return True
    assert self.worstPrefTeam == current_worst, "Worst pref error"
    return False

  def replacingWorstPref(self, newTeam):
    """
    Update the worstPrefs for a section after replacing a team.
    If the new team has a pref of 5, make that the new worstPref.
    Else, iterate through all teams to find the most recent team added
    that has the worstPref.
    """
    if newTeam.currPref == 5:
      self.assignWorstPref(newTeam.currPref, newTeam)
    else:
      for oneTeam in self.teams:
        self.worstPrefCheck(oneTeam)
    assert self.worstPrefTeam in self.teams, "Error resetting worst pref: team listed as worst no longer in list of teams for this section"
    return

  def placeOldTeam(self, oldTeam):
    """
    If a team has been replaced, try to place the team in their next pref.
    If the team was already on their 5th pref, add them to failed allocation
    list.
    """
    oldTeam.currSection = None
    if oldTeam.currPref == 5:
      oldTeam.failed("replaced", self.fail_array)
      assert oldTeam in self.fail_array, "Fail error: team not in " + str(self.fail_array)
      return
    else:
      #print('placing after replace', oldTeam.currPref, oldTeam.preferences, oldTeam.teamid)
      oldTeam.moveToNextSection(self.section_objects, self.classNumber, self.fail_array)
      return

  def replace(self, oldTeam, newTeam):
    """
    Replace a team (oldTeam) in the section that has a worse pref
    with newTeam, then run place on oldTeam and assign the newTeam.
    """
    #adjust section instance attributes
    self.teams = [x for x in self.teams if x.teamid != oldTeam.teamid]
    oldTeam.currentTime = None
    assert oldTeam not in self.teams, "Replace team error: old team not removed"
    self.capacity += 1
    self.numStudents -= oldTeam.size
    self.worstPref = None
    self.worstPrefTeam = None
    assert self.worstPrefTeam == None, "Error resetting worst pref: worstPref not reset to None"

    #assign new team and edit worst prefs
    self.assign(newTeam)
    self.replacingWorstPref(newTeam)

    #place old team in new section
    self.placeOldTeam(oldTeam)
    return

In [ ]:
#sub-class of Section for 295 Sections
class Section295(Section):

  def __init__(self, sectionname, currTime):
    self.classNumber = '295'
    self.fail_array = fail_allocation_295
    self.section_objects = section_295_objects
    cap = 9
    super().__init__(sectionname, cap, currTime)

  def assign(self, team):
    """
    Assign team to this section and update instance variables accordingly.
    """
    #assert team.size <= self.capacity, "Capacity error: Section does not have room"
    assert self.capacity >= 1, "Capacity error: Section does not have room"
    self.teams.append(team)
    assert team in self.teams, "Assignment error: team not added to section's list of teams"
    self.capacity -= 1
    self.numStudents += team.size
    team.currSection = self
    team.currentTime = self.currTime
    assert team.currSection == self, "Assignment error: section not updated as team's current section"
    self.worstPrefCheck(team)

  def place(self, team):
    """
    If team can't fit in section's remaining capacity and the worst preference
    in the section is still better than this team's preference for this section,
    then try to place team in their next pref
    """
    if self.capacity == 0:
      if team.currPref > self.worstPref:
        #if this was their 5th pref, add them to list of teams that
        #couldn't get their top 5.
        if team.currPref >= 5:
          team.currSection = None
          team.failed("place" + self.classNumber, self.fail_array)
          assert team in self.fail_array, "Fail error: team not in " + str(self.fail_array)
        #if not 5th pref, try to place team in their next pref
        else:
          team.moveToNextSection(self.section_objects, self.classNumber, self.fail_array)
      #replace a team in the section that has a worse pref with curr team
      elif team.currPref <= self.worstPref and self.worstPrefTeam.size > team.size:
        self.replace(self.worstPrefTeam, team)
      #if worst team has smaller size than current team, then we can't replace them, as it would still be over capacity,
      #so try to find a diff worst team with size bigger than curr team
      elif team.currPref <= self.worstPref and self.worstPrefTeam.size <= team.size:
        if self.seekOtherWorst(team.size):
          self.replace(self.worstPrefTeam, team)
        elif self.seekOtherWorst(team.size) == False and team.currPref >= 5:
          team.currSection = None
          team.failed("place" + self.classNumber, self.fail_array)
          assert team in self.fail_array, "Fail error: team not in " + str(self.fail_array)
        else:
          team.moveToNextSection(self.section_objects, self.classNumber, self.fail_array)
      else:
        team.moveToNextSection(self.section_objects, self.classNumber, self.fail_array)
    #if team fits in current section, it's in their top 5, and they don't have an assigned
    #section already, assign them
    elif self.capacity >= 1 and team.currPref <= 5:
      if team.currSection != None:
        print("something weird happening", team.teamid)
        return
      else:
        self.assign(team)
        return
    else:
      team.failed("place" + self.classNumber, self.fail_array)
      assert team in self.fail_array, "Fail error: team not in " + str(self.fail_array)
      return


In [ ]:
#sub-class of Section for 270k Sections
class Section270k(Section):

  def __init__(self, sectionname, currTime):
    self.classNumber = '270k'
    self.fail_array = fail_allocation_270k
    self.section_objects = section_270k_objects
    self.teamObjectsFor295 = teams_for_295
    cap = 2
    super().__init__(sectionname, cap, currTime)

  def assign(self, team):
    """
    Assign team to this section and update instance variables accordingly.
    """
    self.teams.append(team)
    self.capacity -= 1
    self.numStudents += team.size
    team.currSection = self
    team.currentTime = self.currTime
    self.worstPrefCheck(team)

  def timeConflictCheck(self, team):
    """
    Returns True if the current section has a time conflict with the given
    team's 295 assigned section. Returns False if there is no conflict.
    """
    for team_295 in self.teamObjectsFor295:
      if team_295.teamid == team.teamid:
        if team_295.currentTime == self.currTime:
          return True
    else:
      return False

  def place(self, team):
    print('capacity: ' + str(self.capacity) + ', currPref: ' + str(team.currPref) + ', worstPref: ' + str(self.worstPref))
    if self.capacity == 0:
      if team.currPref >= 5:
        team.failed("place" + self.classNumber, self.fail_array)
        return
      elif self.timeConflictCheck(team):
        team.moveToNextSection(self.section_objects, self.classNumber, self.fail_array)
        return
      elif team.currPref < 5 and team.currPref < self.worstPref:
        #replace
        self.replace(self.worstPrefTeam, team)
        return
      else:
        assert team.currPref < 5, "Error: moving to next section pref but currPref is >= 5"
        team.moveToNextSection(self.section_objects, self.classNumber, self.fail_array)
        return
    elif self.timeConflictCheck(team):
      team.moveToNextSection(self.section_objects, self.classNumber, self.fail_array)
      return
    elif self.capacity >= 1:
      self.assign(team)
      return
    else:
      team.failed("place" + self.classNumber, self.fail_array)
      assert team in self.fail_array, "Fail error: team not in " + str(self.fail_array)
      return


In [ ]:
#Creating class for each team
class Team:

  def __init__(self, teamid, studentid, size, teamindex):
    self.teamid = teamid #string
    self.studentid = studentid
    self.size = size
    self.teamindex = teamindex

    #Assigned variables
    self.currPref = 1
    self.currSection = None #gets assigned when team has assigned section
    self.preferences = None
    self.currentTime = None

  def createPreferences(self, df):
    """"
    Create an ordered array of 5 numbers that are column indices
    for corresponding section preferences.
    """
    preference_options = ['1st choice', '2nd choice', '3rd choice', '4th choice', '5th choice']
    team_prefs = []
    team_df = df.loc[[self.teamid]]
    for pref in preference_options:
      columns = team_df.loc[:, (team_df == pref).any()].columns
      if columns.size > 0:
        section_name = columns[0]
        team_prefs.append(df.columns.get_loc(section_name))
      else:
        team_prefs.append('None')
    return team_prefs

  def addPref(self):
    """
    Increases team.currPref by 1 to indicate that we have
    moved to the teams' next best preference choice.
    """
    self.currPref += 1

  def moveToNextSection(self, section_objects, classNumber, fail_array):
    """
    Try placing team in their next preferred section.
    """
    self.addPref()
    if self.currPref > 5:
      self.failed("place" + classNumber, fail_array)
      assert self in fail_array, "Fail error: team not in " + str(fail_array)
      return
    temp_index = self.preferences[self.currPref - 1]
    if temp_index == 'None':
      if self.currPref == 5:
        self.failed("place" + classNumber, fail_array)
        assert self in fail_array, "Fail error: team not in " + str(fail_array)
        return
      else:
        self.moveToNextSection(section_objects, classNumber, fail_array)
        return
    section_objects[temp_index].place(self)

  def failed(self, failed_during, failed_array):
    failed_array.append(self)

## Section Matching

### Create section objects

In [ ]:
#using 115 teams since that's how many MEng has
df_295_sections_115 = df_295_sections
special_cases295 = df_295.iloc[0:2]
nonspecial_cases295 = df_295.iloc[2:].sample(frac=1)
df_295_115_teams = pd.concat([special_cases295, nonspecial_cases295])
df_270k_sections_115 = df_270k_sections
special_cases270k = df_270k.iloc[0:2]
nonspecial_cases270k = df_270k.iloc[2:].sample(frac=1)
df_270k_115_teams = pd.concat([special_cases270k, nonspecial_cases270k])

#reset list of objects
section_295_objects = []
section_270k_objects = []

#list of Team objects for teams that did not find a discussion allocation
fail_allocation_295 = []
fail_allocation_270k = []

295 Sections

In [ ]:
#295 sections
section_names_295 = df_295_sections.columns
section_names_270k = df_270k_sections.columns
for i in range(len(section_names_295)):
  temp_name = section_names_295[i]
  temp_regex = re.sub(r"^[^:]*: ", "", temp_name)
  temp_time = re.search(r"^[^:]*", temp_regex).group()
  section_295_objects.append(Section295(temp_name, temp_time))

In [ ]:
#295 teams

#array of all team objects for 295 section matching
teams_for_295 = []

for i in np.arange(len(df_295_115_teams)):
  temp_teamid = df_295_115_teams.index[i]
  studentid = df_295_115_teams.iloc[i][4]
  size = int(df_295_115_teams.iloc[i][5])
  new_team = Team(temp_teamid, studentid, size, i)
  new_team.preferences = new_team.createPreferences(df_295_sections)
  first_pref_index = new_team.preferences[0]
  teams_for_295.append(new_team)
  section_295_objects[first_pref_index].place(new_team)

270k Sections

In [ ]:
section_names_270k

Index(['For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101A: TUES 10:00 - 10:30am; Instructor: A. Balaraman]',
       'For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101B: TUES 10:30 - 11:00am; Instructor: A. Balaraman]',
       'For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101C: TUES 11:30am - 12:00pm; Instructor: A. Balaraman]',
       'For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101D: TUES 12:00 - 12:30pm; Instructor

In [ ]:
#270k sections
for i in range(len(section_names_270k[0:56])):
  temp_name = section_names_270k[i]
  temp_regex = re.sub(r"^[^:]*: ", "", temp_name)
  temp_time = re.search(r"^[^:]*", temp_regex).group()
  temp_section = Section270k(temp_name, temp_time)
  temp_section.teamObjectsFor295  = teams_for_295
  section_270k_objects.append(temp_section)

In [ ]:
df_270k_sections.iloc[:, :56]

,"For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101A: TUES 10:00 - 10:30am; Instructor: A. Balaraman]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101B: TUES 10:30 - 11:00am; Instructor: A. Balaraman]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101C: TUES 11:30am - 12:00pm; Instructor: A. Balaraman]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 101D: TUES 12:00 - 12:30pm; Instructor: A. Balaraman]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 102A: FRI 12:00 - 12:30pm; Instructor: P. Deb]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 102B: FRI 12:30 - 1:00pm; Instructor: P. Deb]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 102C: FRI 1:00 - 1:30pm; Instructor: P. Deb]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 102D: FRI 1:30 - 2:00pm; Instructor: P. Deb]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 103A: TUES 2:30 - 3:00pm; Instructor: L. Edwards]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [IN-PERSON Section 103B: TUES 3:00 - 3:30pm; Instructor: L. Edwards]",...,"For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 111C: TUES 9:30 - 10:00am; Instructor: K. Tandowsky]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 111D: TUES 10:00 - 10:30am; Instructor: K. Tandowsky]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 111E: TUES 11:00 - 11:30am; Instructor: K. Tandowsky]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 111F: TUES 11:30am- 12:00pm; Instructor: K. Tandowsky]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 111G: TUES 12:00 - 12:30pm; Instructor: K. Tandowsky]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through FIFTH section preferences for various date/time options (first is most desired). [ONLINE Section 111H: TUES 12:30- 1:00pm; Instructor: K. Tandowsky]","For ENGIN270K - Coaching for High Performance Teams, please select your team's FIRST through 

In [ ]:
#270k teams

#array of all team objects for 270k section matching
teams_for_270k = []

for i in np.arange(len(df_270k_115_teams)):
  temp_teamid = df_270k_115_teams.index[i]
  studentid = df_270k_115_teams.iloc[i][4]
  size = int(df_270k_115_teams.iloc[i][5])
  new_team = Team(temp_teamid, studentid, size, i)
  new_team.preferences = new_team.createPreferences(df_270k_sections.iloc[:, :56])
  first_pref_index = new_team.preferences[0]
  teams_for_270k.append(new_team)
  section_270k_objects[first_pref_index].place(new_team)

capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 1, currPref: 1, worstPref: 1
capacity: 1, currPref: 1, worstPref: 1
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 0, currPref: 1, worstPref: 1
capacity: 1, currPref: 2, worstPref: 1
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 1, currPref: 1, worstPref: 1
capacity: 2, currPref: 1, worstPref: None
capacity: 0, currPref: 1, worstPref: 1
capacity: 2, currPref: 2, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 2, currPref: 1, worstPref: None
capacity: 0, currPref: 1, worstPref: 1
capacity: 1, 

## Tests

### 295 Tests

In [ ]:
temp_teamids = []
for section in section_295_objects:
  for team in section.teams:
    temp_teamids.append(team.teamid)

sum(pd.Series(temp_teamids).value_counts() > 1)

0

In [ ]:
count_teams = 0
for section in section_295_objects:
  count_teams += len(section.teams)
count_teams

107

In [ ]:
len(fail_allocation_295)

8

### 270k Tests

In [ ]:
temp_teamids2 = []
for section in section_270k_objects:
  for team in section.teams:
    temp_teamids2.append(team.teamid)

sum(pd.Series(temp_teamids2).value_counts() > 1)

0

In [ ]:
count_teams2 = 0
for section in section_270k_objects:
  count_teams2 += len(section.teams)
count_teams2

91

In [ ]:
len(fail_allocation_270k)

24

### Teams that fail in both

In [ ]:
fail_allocation_both = []
for i in fail_allocation_270k:
  for j in fail_allocation_295:
    if i.teamid == j.teamid:
      fail_allocation_both.append(i)
len(fail_allocation_both)

2

### Verifying no time conflict:¶

In [ ]:
tc = 0
for i in teams_for_295:
  for j in teams_for_270k:
    if i.teamid == j.teamid:
      if i.currentTime == j.currentTime:
        if i.currentTime:
          print(i.currentTime)
          tc += 1
tc

0

## Export Results

In [ ]:
# what pref each team got assigned

prefs_assigned = []
for section in section_295_objects:
  for team in section.teams:
    prefs_assigned.append(team.currPref)

In [ ]:
# what section each team got assigned 295

all_teams295 = []
all_teams270k = []
sections_assigned295 = []
sections_assigned270k = []

for section in section_295_objects:
  for team in section.teams:
    all_teams295.append(team.teamid)
    renamed_section_295 = re.sub(r'(.*\[)|ONLINE|IN-PERSON|:.*', ' ', team.currSection.name)
    sections_assigned295.append(renamed_section_295)


for team in fail_allocation_295:
  all_teams295.append(team.teamid)
  sections_assigned295.append('None')

In [ ]:
# what section each team got assigned 295

all_teams295 = []
all_teams270k = []
sections_assigned295 = []
sections_assigned270k = []

for section in section_295_objects:
  for team in section.teams:
    all_teams295.append(team.teamid)
    renamed_section_295 = re.sub(r'(.*\[)|ONLINE|IN-PERSON|:.*', ' ', team.currSection.name)
    sections_assigned295.append(renamed_section_295)


for team in fail_allocation_295:
  all_teams295.append(team.teamid)
  sections_assigned295.append('None')
# what section each team got assigned 270k

for section in section_270k_objects:
  for team in section.teams:
    all_teams270k.append(team.teamid)
    renamed_section_270k = re.sub(r'(.*\[)|ONLINE|IN-PERSON|:.*', ' ', team.currSection.name)
    sections_assigned270k.append(renamed_section_270k)


for team in fail_allocation_270k:
  all_teams270k.append(team.teamid)
  sections_assigned270k.append('None')

In [ ]:
#create df of each team and their assigned section for 295

df_assigned_sections295 = pd.DataFrame(data={'teamid':all_teams295, '295_assigned_section':sections_assigned295})
df_assigned_sections295

,teamid,295_assigned_section
0,175A,Section 3
1,162A,Section 3
2,141,Section 3
3,138B,Section 3
4,181D,Section 3
...,...,...
110,071B,None
111,146A,None
112,154,None
113,224G FINTECH,None


In [ ]:
#create df of each team and their assigned section for 270k

df_assigned_sections270k = pd.DataFrame(data={'teamid':all_teams270k, '270k_assigned_section':sections_assigned270k})
df_assigned_sections270k

,teamid,270k_assigned_section
0,072A,Section 101A
1,155,Section 101A
2,181B,Section 101B
3,217,Section 101C
4,158,Section 101C
...,...,...
110,071A,None
111,125A,None
112,085,None
113,162B,None


In [ ]:
len(df_assigned_sections270k['270k_assigned_section'].unique())

52

In [ ]:
final_df = df_assigned_sections295.merge(df_assigned_sections270k, how = 'inner')
final_df.to_csv('final.csv')

## Section Numbers of Students and Teams

In [ ]:
sectionnames295 = []
section_amounts_295 = []
section_team_amounts_295 = []
for section in section_295_objects:
  renamed_section295 = re.sub(r'(.*\[)|ONLINE|IN-PERSON|:.*', ' ', section.name)
  sectionnames295.append(renamed_section295)
  section_amounts_295.append(section.numStudents)
  section_team_amounts_295.append(len(section.teams))

In [ ]:
df_section_amounts_295 = pd.DataFrame(data={'section name':sectionnames295, 'number of students':section_amounts_295, 'number of teams':section_team_amounts_295})
df_section_amounts_295

,section name,number of students,number of teams
0,Section 3,36,9
1,Section 4,27,9
2,Section 5,19,5
3,Section 6,19,5
4,Section 7,29,9
5,Section 8,36,9
6,Section 9,24,7
7,Section 10,32,9
8,Section 11,37,9
9,Section 12,22,6


In [ ]:
sectionnames270k = []
section_amounts_270k = []
section_team_amounts_270k = []

for section in section_270k_objects:
  renamed_section270k = re.sub(r'(.*\[)|ONLINE|IN-PERSON|:.*', ' ', section.name)
  sectionnames270k.append(renamed_section270k)
  section_amounts_270k.append(section.numStudents)
  section_team_amounts_270k.append(len(section.teams))

In [ ]:
df_section_amounts_270k = pd.DataFrame(data={'section name':sectionnames270k, 'number of students':section_amounts_270k, 'number of teams':section_team_amounts_270k})
df_section_amounts_270k

,section name,number of students,number of teams
0,Section 101A,6,2
1,Section 101B,3,1
2,Section 101C,6,2
3,Section 101D,10,2
4,Section 102A,8,2
5,Section 102B,7,2
6,Section 102C,0,0
7,Section 102D,3,1
8,Section 103A,10,2
9,Section 103B,3,1


In [ ]:
len(section_270k_objects)

56